# DES Y3 2x2pt Likelihood for LCDM Model v1

**Note: BayesFast is still undocumented and under development by He Jia at https://github.com/HerculesJack/bayesfast. James Sullivan (jmsullivan@berkeley.edu) modified this notebook for Y3 2x2pt.**

### Setup:

In [1]:
import os
os.environ['OMP_NUM_THREADS'] = '1' 

Initializing the [dask](https://distributed.dask.org/en/latest/) client. 
Here we have one node with 64 cores. 
See [this](https://jobqueue.dask.org/en/latest/configurations.html#nersc-cori)
for examples on how to start the client using multiple nodes on NERSC.

In [2]:
from distributed import Client, LocalCluster
cluster = LocalCluster(n_workers=64, threads_per_worker=1)
client = Client(cluster)
client

Client Scheduler: tcp://127.0.0.1:37959 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 64 Cores: 64 Memory: 540.15 GB


### User Inputs:

Add your chain here if you want to compare:

In [3]:
import numpy as np

#if you have a chain you want to check, put it here and set to True
chain_supplied=False
your_chain_path = ' '

#get number of parameters and comparison chain if necessary
if(not chain_supplied):
    #example multinest chain - only used to get number of parameters if you don't supply a chain
    cfile ='./y3_notebook_v1/mn_inifiles/chain_2x2pt_fiducial_scale_cut_test_nl_bias_baryon.fits_scales_2x2pt_12_6.ini_lcdm.txt'
    nParams = np.loadtxt(cfile).shape[1]-5
else:
    your_chain = np.loadtxt(your_chain_path)
    nParams = your_chain.shape[1]-5
    chain_x_mn,chain_p_mn = your_chain[:,:nParams].copy(),your_chain[:, -1].copy() 

Decide whether to use importance sampling, and if so how many samples:

In [4]:
useIS=True
n_IS = 2000

### Reading ini files:

Here we get the parameter ranges and start values for the parameters which are not fixed

In [5]:
#file reading to get the priors, ranges, and init values 
priors,values=[],[]
pkeys,vkeys=[],[]

def is_number(string):
    try:
        float(string)
        return True
    except:
        return None



with open('./values.ini') as f:
    for i,line in enumerate(f):
        if('=' in line and '#' not in line and ';' not in line):
            line_values=[]
            if(i<=4 ): #\t is the separator instead of space for some reason
                pre_num_list = line[:-1].split('=')[1].split('\t')
            else:
                pre_num_list = line[:-1].split('=')[1].split(' ')
            for substr in pre_num_list: 
                if(is_number(substr)): 
                    line_values.append(float(substr))
            if(len(line_values)>1): #ignore fixed parameters
                if('alpha1' in line or 'alpha2' in line): #spacing is different for alpha
                    vkeys.append(line.split('=')[0])
                else:
                    vkeys.append(line.split(' ')[0]) 
                values.append(line_values)
                        
#all gaussian
with open('./priors.ini') as f:
    for line in f:
        if('=' in line and '#' not in line and ';' not in line and 'a_planck' not in line):
            line_priors=[]
            pkeys.append(line.split(' ')[0])
            pre_num_list = line[:-1].split('=')[1].split(' ')
            for substr in pre_num_list: 
                if(is_number(substr)): 
                    line_priors.append(float(substr))
            priors.append(line_priors)    
            
    

init_values = dict(zip(vkeys,[v[1] for v in values]))
ranges = dict(zip(vkeys,[[v[0],v[2]] for v in values]))

init_mu = np.fromiter(init_values.values(),dtype=float)
para_range = np.asarray([[v[0],v[2]] for v in values])


In [6]:
#now for y3
init_sig = (para_range[:, 1] - para_range[:, 0]) / 1000
nParams = len(init_mu)
print('parameters are', vkeys)
idx_dict = dict(zip(vkeys,np.arange(len(values))))
_nonlinear_indices = np.array([idx_dict['omega_m'],
                               idx_dict['h0'],
                               idx_dict['omega_b'],
                               idx_dict['n_s'],
                               idx_dict['A_s'],
                               idx_dict['A1'],
                               idx_dict['A2'],
                               idx_dict['alpha1'],
                               idx_dict['alpha2'],
                               idx_dict['bias_ta']
                              ])

_constrained_indices = np.array([
                                idx_dict['omega_m'],
                                idx_dict['A_s'],
                                idx_dict['A1'],
                                idx_dict['A2'],
                                idx_dict['alpha1'],
                                idx_dict['alpha2'],
                                idx_dict['bias_ta']
                                ])

parameters are ['omega_m', 'h0', 'omega_b', 'n_s', 'A_s', 'm1', 'm2', 'm3', 'm4', 'bias_1', 'bias_2', 'bias_3', 'bias_4', 'bias_5', 'b1', 'b2', 'b3', 'b4', 'b5', 'A1', 'A2', 'alpha1', 'alpha2', 'bias_ta']


In [7]:
print('nonlinear: ',np.asarray(vkeys)[_nonlinear_indices])
print('constrained: ',np.asarray(vkeys)[_constrained_indices])

nonlinear:  ['omega_m' 'h0' 'omega_b' 'n_s' 'A_s' 'A1' 'A2' 'alpha1' 'alpha2'
 'bias_ta']
constrained:  ['omega_m' 'A_s' 'A1' 'A2' 'alpha1' 'alpha2' 'bias_ta']


### Cosmosis pipeline:

Initializing the cosmosis pipeline.
We only use cosmosis to compute the models (2pt functions),
which are approximated by polynomial surrogates during sampling.

In [8]:
#3xpt readme instructions for running 2x2pt chain, picked a random one
os.environ['DATAFILE'] = 'scale_cut_test_nl_bias_baryon.fits'
os.environ['DEMODEL'] = 'lcdm'
os.environ['SCALE_CUTS']='scales_2x2pt_12_6.ini'
os.environ['DATASET']='2x2pt'
os.environ['RUN_NAME_STR']='y3-test-bayesfast'
os.environ['OUTDIR']='${SCRATCH}'+'/chains'

In [9]:
from cosmosis.runtime.config import Inifile
from cosmosis.runtime.pipeline import LikelihoodPipeline
import sys

old_stdout = sys.stdout
sys.stdout = open(os.devnull, 'w')
ini_string='./2x2pt_fiducial/params.ini'
ini = Inifile(ini_string)
pipeline = LikelihoodPipeline(ini)
sys.stdout = old_stdout

KeyboardInterrupt: 

In [ ]:
start = pipeline.start_vector()

In [ ]:
results = pipeline.run_results(start)

In [ ]:
from scipy.stats import norm
from scipy.linalg import sqrtm

#gaussian priors
_prior_indices = np.array([idx_dict['bias_1'],
                           idx_dict['bias_2'],
                           idx_dict['bias_3'],
                           idx_dict['bias_4'],
                           idx_dict['bias_5'],
                           idx_dict['m1'],
                           idx_dict['m2'],
                           idx_dict['m3'],
                           idx_dict['m4'],
                            ])

_flat_indices = np.setdiff1d(np.fromiter(idx_dict.values(),dtype=int),_prior_indices)

_prior_mu = np.asarray(priors)[:,0]
_prior_sig = np.asarray(priors)[:,1]


_prior_norm = (
    -0.5 * np.sum(np.log(2 * np.pi * _prior_sig**2)) - np.sum(np.log(
    norm.cdf(para_range[_prior_indices, 1], _prior_mu, _prior_sig) -
    norm.cdf(para_range[_prior_indices, 0], _prior_mu, _prior_sig))) - 
    np.sum(np.log(para_range[_flat_indices,1] - para_range[_flat_indices,0])) 
              )

_d = results.block['data_vector', '2pt_data']
nData = _d.shape[0]
print('nData',nData)

#Note that this will not work if pm_marg=True
_invC = results.block['data_vector', '2pt_inverse_covariance']

_invC_r = sqrtm(_invC)

_d_diag = _d @ _invC_r 
_norm = results.block['data_vector', '2pt_norm'] 

def des_prior_f(x): #prior chisq + log prior
    chi2 = -0.5 * np.sum(((x[_prior_indices] - _prior_mu) / _prior_sig)**2)
    return chi2 + _prior_norm

def des_prior_j(x): #prior gradient
    foo = np.zeros((1, nParams))
    foo[0, _prior_indices] = -(x[_prior_indices] - _prior_mu) / _prior_sig**2
    return foo

def des_2pt_theory(x,_invC_r=_invC_r):
    #run DES pipeline to get data*invCov , theory *invCov
    try:
        import os, sys
        os.environ['OMP_NUM_THREADS'] = '1'
        os.environ['DATAFILE'] = 'scale_cut_test_nl_bias_baryon.fits'
        os.environ['DEMODEL'] = 'lcdm'
        os.environ['SCALE_CUTS']='scales_2x2pt_12_6.ini'
        os.environ['DATASET']='2x2pt'
        os.environ['RUN_NAME_STR']='y3-test-bayesfast-broken'
        os.environ['OUTDIR']='${SCRATCH}'+'/chains'
        from cosmosis.runtime.config import Inifile
        from cosmosis.runtime.pipeline import LikelihoodPipeline
        ini_string='./2x2pt_fiducial/params.ini'
        old_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')
        ini = Inifile(ini_string)
        pipeline = LikelihoodPipeline(ini)
        sys.stdout = old_stdout
        res = pipeline.run_results(x)
        rres = res.block['data_vector', '2pt_theory'] @ _invC_r
        return rres
    except:
        print('Failed to run pipeline! Returning nans')
        return np.nan*np.ones(nData)

def chi2_f(m): #lhood chisq, uses covariance now
    return np.atleast_1d(-0.5 * np.sum((m - _d_diag)**2) + _norm)

def chi2_fj(m): #lood chisq gradient
    return (np.atleast_1d(-0.5 * np.sum((m - _d_diag)**2) + _norm), 
            -(m - _d_diag)[np.newaxis])

def des_post_f(like, x): #like+prior
    return like + des_prior_f(x)

def des_post_fj(like, x): #like + prior and prior gradients
    return like + des_prior_f(x), np.concatenate(
        (np.ones((1, 1)), des_prior_j(x)), axis=-1)


### BayesFast Setup:

BayesFast is not documented yet. Below is a brief note on its usage.
Not all the functionality is used in this notebook.

* `Module` : analogous to cosmosis modules, with optional analytic Jacobian. 
  * ```
    __init__(self, fun=None, jac=None, fun_and_jac=None,
             input_vars=['__var__'], output_vars=['__var__'],
             copy_vars=None, paste_vars=None, delete_vars=None,
             recombine_input=False, recombine_output=False,
             var_scales=None, label=None, fun_args=(), fun_kwargs={},
             jac_args=(), jac_kwargs={}, fun_and_jac_args=(),
             fun_and_jac_kwargs={})
    ```
  * You may define its `fun` and/or `jac` and/or `fun_and_jac`,
    and call them with `Module.fun` etc.
    When `Module.fun` is called, we will first check if you have defined its `fun`.
    If not, we will check if you have defined its `fun_and_jac`.
    If still not, an exception will be raised. Similar for `Module.jac` and `Module.fun_and_jac`.
  * You need to specify the name(s) of `input_vars` and `output_vars`
    as a list of strings, or a string if there is only one variable.
    This will be used to track the variables during the evaluation of the pipeline.
    All the variables should be stored and used as 1-d numpy arraies.
  * Let's say we have a `Module` with `input_vars` A and B, whose shapes are `(a,)` and `(b,)`.
    While the `output_vars` are C and D, whose shapes are `(c,)` and `(d,)`.
    Then the signature of its `fun` should be `(a,),(b,)->(c,),(d,)`.
    The signature of its `jac` should be `(a,),(b,)->(c,a+b),(d,a+b)`.
    The signature of its `fun_and_jac` should be `(a,),(b,)->((c,),(d,)),((c,a+b),(d,a+b))`.
  * For convenience, you can also use the arguments `recombine_input` and `recombine_output`.
    For the example above, if `recombine_input` is True, 
    the input of `fun` should have shape `(a+b,)`.
    Assuming `a+b=e+f`, if `recombine_input` is `(e,f)`, 
    the input should have shape `(e,f)`. Similar for `recombine_output`.

In [ ]:
import bayesfast as bf
m_0 = bf.Module(fun=des_2pt_theory, input_vars='x', #parameters-> theory model
                output_vars='m')
m_1 = bf.Module(fun=chi2_f, fun_and_jac=chi2_fj, #theory model -> likelihood 
                input_vars='m', output_vars='like')
m_2 = bf.Module(fun=des_post_f, fun_and_jac=des_post_fj, #likelihood and parameters -> log posterior
                input_vars=['like', 'x'], output_vars='logp')

* `Density`: derived from `Pipeline`, analogous to cosmosis LikelihoodPipeline.
  * ```
    __init__(self, module_list=None, input_vars=['__var__'], var_dims=None,
             surrogate_list=None, var_scales=None, hard_bounds=False)
    ```
  * The overall input of `Density` should be a single array, 
    and you need to tell us how to split it using `input_vars` and `var_dims`.
  * `var_scales` should have shape `(input_size, 2)`. 
    If None, it will be set as `((0,1),(0,1)...)`.
  * If `hard_bounds` is False, 
    we only linearly rescale it by mapping `var_scales` to `((0,1),(0,1)...)`.
    If `hard_bounds` is True,
    we will transform it to `((-inf,+inf),(-inf,+inf)...)` using nonlinear transformation.
    You can also set it separately for each variable 
    by using an array with shape `(input_size, 2)` for `hard_bounds`.

In [ ]:
d_0 = bf.Density(density_name='logp', module_list=[m_0, m_1, m_2], #stack modules to go from params -> log posterior
                 input_vars='x', var_dims=nParams, var_scales=para_range, 
                 hard_bounds=True)
d_0(start), results.post

* `PolyConfig`: used to config `PolyModel`.
  * ```
    __init__(self, order, input_mask=None, output_mask=None, coef=None)
    ```
  * `order` should be one of `('linear', 'quadratic', 'cubic-2', 'cubic-3')`, 
    where `cubic-2` means cubic model without 'xyz' terms.
  * If you only want to define it on some of the input (output) variables,
    you can use `input_mask` (`output_mask`).


* `Surrogate`: derived from `Module`.
  * ```
    __init__(self, input_size=None, output_size=None, scope=(0, 1),
             input_vars=['__var__'], output_vars=['__var__'],
             copy_vars=None, paste_vars=None, delete_vars=None,
             recombine_input=True, *args, **kwargs)
    ```
  * `scope`: `(start,extent)`, e.g. `(0,1)` means it will replace #0 `Module` in `module_list`.


* `PolyModel`: derived from `Surrogate`.
  * ```
    __init__(self, configs, *args, **kwargs)
    ```
  * `configs` should be a `PolyConfig` or a list of them. 
    Or you can also just use its `order` if you don't need to set the masks.
    In this case, for example, `'quadratic'` will be interpreted as `('linear','quadratic')`.

Here, during optimization, we use 24-d linear model.
During sampling, we use 24-d linear plus 9-d quadratic model.?

In [ ]:
print('Checking size of polynomial inputs. There are {0} model paramters and {1} data points'.format(nParams,nData))
s_0 = bf.modules.PolyModel('linear', nParams, nData, input_vars='x', #approximate theory model with linear model
                           output_vars='m', var_scales=para_range)
pc_0 = bf.modules.PolyConfig('linear') #another linear model
pc_1 = bf.modules.PolyConfig('quadratic', input_mask=_nonlinear_indices) #nonlinear model - quadratic
s_1 = bf.modules.PolyModel([pc_0, pc_1], nParams, nData, input_vars='x', #approximate theory model as linear + quadratic
                           output_vars='m', var_scales=para_range)


`alpha_n=2` means we use twice the minimum number of samples required by fitting the surrogate model.

We iterate the block quadratic model for two steps, and in the end, 
we use truncated importance sampling with n=2000 samples, while the weights w are truncated at < w >n^0.25.

At the beginning, you need to provide a bunch of `x_0` to fit the initial surrogate model.

In [ ]:
#check if in bounds provided by ranges in values.ini
def _in_bound(xx, bound):
    xxt = np.atleast_2d(xx).T
    return np.product([np.where(xi>bound[i,0], True, False) * 
                       np.where(xi<bound[i,1], True, False) for i, xi in 
                       enumerate(xxt)], axis=0).astype(bool)


#setting up steps in recipe
opt_0 = bf.recipe.OptimizeStep(s_0, alpha_n=2,hmc_options={"sampler_options":{"max_treedepth":5}
                                                           "n_iter":1000,
                                                           "n_warmup":200}) #linear model for optimizer
sam_0 = bf.recipe.SampleStep(s_1, alpha_n=2, reuse_steps=1,  #linear+quadratic sample step
                             fit_options={'use_mu_f': True},sample_options={"sampler_options":{"max_treedepth":5},
                                                                            "n_iter":1000,
                                                                            "n_warmup":200})
sam_1 = bf.recipe.SampleStep(s_1, alpha_n=2, reuse_steps=1, #linear+quadratic sample step
                             fit_options={'use_mu_f': True},sample_options={"sampler_options":{"max_treedepth":8},
                                                                            "n_iter":1000,
                                                                            "n_warmup":200})
#surrogate model training points
x_0 = bf.utils.random.multivariate_normal(init_mu, np.diag(init_sig**2), 100)
x_0 = x_0[_in_bound(x_0, para_range)] #checking that points are inside bounds

if(not useIS):
    r_0 = bf.recipe.Recipe(density=d_0, client=client, optimize=opt_0, 
                       sample=[sam_0, sam_1],
                       x_0=x_0, 
                       random_state=1)
else:
    pos_0 = bf.recipe.PostStep(n_is=n_IS, k_trunc=0.25) #post step - importance sampling

    r_0 = bf.recipe.Recipe(density=d_0, client=client, optimize=opt_0, #putting it all together
                           sample=[sam_0, sam_1],
                           post=pos_0, 
                           x_0=x_0, 
                           random_state=1)


### Running Bayesfast:

In [ ]:
import time
time.strftime('%H:%M%p %Z on %b %d, %Y')

In [ ]:
r_0.run()

In [ ]:
time.strftime('%H:%M%p %Z on %b %d, %Y')

In [ ]:
r_0.get()._fields

### Plot Results:

In [ ]:
#plots - bayesfast linear+quadratic approx to posterior (with and without importance sampling)
%matplotlib inline
from getdist import plots, MCSamples
import matplotlib.pyplot as plt
labels = ['\\Omega_m', 
          'A_s', 
          'A_{1, \\rm IA}', 
          'A_{2, \\rm IA}',
          '\\alpha_{1, \\rm IA}',
          '\\alpha_{2, \\rm IA}',
          'b_{\\rm ta}'
         ] 
names = ["x%s"%i for i in range(len(labels))]
n_CALL = r0.n_call

In [ ]:
if(chain_supplied):
    s_mn = MCSamples(
        samples=chain_x_mn[:, _constrained_indices], weights=chain_p_mn, names=names, 
        labels=labels, ranges=dict(zip(names, para_range[_constrained_indices])), 
        label='MultiNest: original model ({0} iterations)'.format(chain_p_mn.shape[0]))
    
s_bf = MCSamples(
    samples=r_0.get().samples[:, _constrained_indices], names=names, 
    labels=labels, ranges=dict(zip(names, para_range[_constrained_indices])), 
    label='BayesFast 5-5-8: block quadratic model ({0} calls)'.format(n_CALL-n_IS))

s_bf_i = MCSamples(
    samples=r_0.get().samples[:, _constrained_indices],
    weights=r_0.get().weights, names=names, 
    labels=labels, ranges=dict(zip(names, para_range[_constrained_indices])), 
    label='BayesFast 5-5-8: block quadratic model with IS ({0} calls)'.format(n_CALL))
g = plots.getSubplotPlotter()
g.settings.figure_legend_loc = 'upper right'
g.settings.axes_fontsize = 14
g.settings.lab_fontsize = 16
g.settings.legend_fontsize = 15
g.settings.lw_contour = 2
g.settings.lw1 = 2
if(chain_supplied):
    g.triangle_plot([ s_mn, s_bf, s_bf_i], filled=False, contour_args={'alpha':1}, 
                    diag1d_kwargs={'normalized':True}, contour_colors=[ 'dodgerblue',
                    'gold', 'red'])
else:
    g.triangle_plot([s_bf, s_bf_i], filled=False, contour_args={'alpha':1}, 
                    diag1d_kwargs={'normalized':True}, contour_colors=['gold', 'red'])
plt.tight_layout()
#plt.savefig('y3_bf.png')
plt.show()

distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting